# Verificando se a instalacão está correta


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome(executable_path='C:/chromedriver-win64/chromedriver.exe')

driver.get("https://www.selenium.dev/selenium/web/web-form.html")

title = driver.title

driver.implicitly_wait(0.5)

text_box = driver.find_element(by=By.NAME, value="my-text")
submit_button = driver.find_element(by=By.CSS_SELECTOR, value="button")

text_box.send_keys("Selenium")
submit_button.click()

message = driver.find_element(by=By.ID, value="message")
text = message.text

print(text)

driver.quit()

## Verificando versão do selenium

In [ ]:
#!pip --upgrade selenium

In [ ]:
!pip show selenium

In [ ]:
#pip install --upgrade selenium

## Abrindo página do quinto andar e fechando

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# Caminho para o seu chromedriver
caminho_driver = 'C:/chromedriver-win64/chromedriver.exe'

# Configurações do navegador (opcional, mas recomendável)
chrome_options = Options()
chrome_options.add_argument("--start-maximized")

# Iniciando o serviço do ChromeDriver
servico = Service(executable_path=caminho_driver)
driver = webdriver.Chrome(service=servico, options=chrome_options)

# Acessar o site
url = "https://www.quintoandar.com.br/"
driver.get(url)

# Esperar um pouco para ver se carregou
time.sleep(5)

# Finalizar (por enquanto)
driver.quit()

## Versão final

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import time
start_time = time.time()

# Configuração do driver do Chrome
caminho_driver = 'C:/chromedriver-win64/chromedriver.exe'
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
servico = Service(executable_path=caminho_driver)
driver = webdriver.Chrome(service=servico, options=chrome_options)

# Funções para realizar as interações e extração dos dados

def clicar_botao_buscar(driver):
    # Clica no botão "Buscar"
    botao_buscar = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[input-latency="landing-search-button"]'))
    )
    botao_buscar.click()
    time.sleep(5)

def clicar_pular_tudo(driver):
    try:
        pular_tudo = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '//span[text()="Pular tudo"]'))
        )
        pular_tudo.click()
        time.sleep(2)
        print("✅ Clicou em 'Pular tudo'")
    except Exception as e:
        print("⚠️ Botão 'Pular tudo' não apareceu (ou já foi pulado).", e)

def digitar_e_selecionar_bairro(driver, nome_bairro):
    # Digita e seleciona o bairro no input
    input_bairro = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "downshift-1-input"))
    )
    input_bairro.clear()
    input_bairro.send_keys(nome_bairro)
    time.sleep(1)  # Aguarda as sugestões aparecerem
    xpath_opcao = f'//span[text()="{nome_bairro}"]'
    sugestao = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, xpath_opcao))
    )
    sugestao.click()
    time.sleep(2)

def extrair_imoveis(driver, limite=30):
    imoveis = []
    
    while len(imoveis) < limite:
        # Aguarda os cards de imóvel aparecerem usando o atributo "data-testid"
        WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.XPATH, '//div[@data-testid="house-card-container"]'))
        )
        time.sleep(2)
        cards = driver.find_elements(By.XPATH, '//div[@data-testid="house-card-container"]')
        
        for idx, card in enumerate(cards):
            if len(imoveis) >= limite:
                break

            # Imprime o status antes de coletar o imóvel
            print(f"Coletando informações do imóvel {len(imoveis)+1}/{limite}...")
            
            try:
                # Extrai o URL do anúncio
                link_element = card.find_element(By.XPATH, './/a')
                url_anuncio = link_element.get_attribute("href")
                
                # Extrai a descrição
                descricao = card.find_element(By.XPATH, './/h2[contains(@class, "UQvm9e")]').text

                # Extrai o preço
                preco = card.find_element(By.XPATH, './/div[contains(@class, "Cozy__CardTitle-Title")]//p').text

                # Extrai o condomínio/IPTU
                condominio = card.find_element(By.XPATH, './/div[contains(@class, "Cozy__CardTitle-Subtitle")]//p').text

                # Extrai as informações gerais (ex.: área, quartos, vagas)
                info_geral = card.find_element(By.XPATH, './/h3[contains(@class, "o6Ojuq")]').text

                # Extrai o endereço
                endereco = card.find_element(By.XPATH, './/h2[contains(@class, "Ci-jp3")]').text

                imoveis.append({
                    'URL do anúncio': url_anuncio,
                    'Descrição': descricao,
                    'Preço': preco,
                    'Condomínio/IPTU': condominio,
                    'Info Geral': info_geral,
                    'Endereço': endereco
                })
            except Exception as e:
                print(f"⚠️ Erro ao coletar imóvel {len(imoveis)+1}: {e}")
                continue
        
        # Se ainda não atingiu o limite, tenta clicar em "Ver mais"
        if len(imoveis) < limite:
            try:
                botao_ver_mais = WebDriverWait(driver, 1).until(
                    EC.element_to_be_clickable((By.ID, "see-more"))
                )
                botao_ver_mais.click()
                time.sleep(1)
            except Exception as e:
                print("⚠️ Botão 'Ver mais' não encontrado. Finalizando.", e)
                break

    return pd.DataFrame(imoveis)

# --- Início do fluxo principal ---

print("\nDigite corretamente os nomes da cidade e (opcional) bairro")
cidade = input("Digite a cidade: ").strip()
bairro = input("Digite o bairro (deixe em branco para pesquisar por cidade inteira): ").strip()

# Adiciona input para quantos imóveis o usuário deseja pesquisar
limite_imoveis_input = input("Digite quantos imóveis deseja pesquisar: ").strip()
try:
    limite_imoveis = int(limite_imoveis_input)
except ValueError:
    print("Entrada inválida para o número de imóveis. Usando 30 como padrão.")
    limite_imoveis = 30

# --- Inicia a navegação ---

driver.get("https://www.quintoandar.com.br/")
time.sleep(3)  # Aguarda a página carregar

# Clica no botão "Comprar"
botao_comprar = driver.find_element(By.XPATH, '//button[.//span[text()="Comprar"]]')
botao_comprar.click()

# Preenche o campo de cidade
campo_cidade = driver.find_element(By.ID, "downshift-2-input")
campo_cidade.clear()
campo_cidade.send_keys(cidade)

# Se o bairro foi informado, preenche o campo de bairro; caso contrário, prossegue somente com a cidade
if bairro:
    campo_bairro = driver.find_element(By.ID, "downshift-3-input")
    campo_bairro.clear()
    campo_bairro.send_keys(bairro)
else:
    print("🔍 Pesquisa realizada somente pela cidade, sem filtro de bairro.")

# Clica no botão de buscar e pula pop-ups se necessário
clicar_botao_buscar(driver)
clicar_pular_tudo(driver)

# Se o bairro foi informado, atualiza-o na nova página; se não, pula essa etapa
if bairro:
    digitar_e_selecionar_bairro(driver, bairro)
else:
    print("🔔 Pulando atualização de bairro, pesquisa por cidade inteira.")

# Extrai os imóveis usando o limite informado pelo usuário
df_imoveis = extrair_imoveis(driver, limite=limite_imoveis)

# Exibe os dados extraídos
print("\nExtração concluída com sucesso!:")
tempo_total = time.time() - start_time
print(f"\n⏱️ Tempo total de execução: {tempo_total:.2f} segundos")

# Aguarda para visualizar os resultados e encerra o driver, se desejado
time.sleep(5)
driver.quit()


Digite corretamente os nomes da cidade e (opcional) bairro
Digite a cidade: São Paulo
Digite o bairro (deixe em branco para pesquisar por cidade inteira): 
Digite quantos imóveis deseja pesquisar: 1000
🔍 Pesquisa realizada somente pela cidade, sem filtro de bairro.
✅ Clicou em 'Pular tudo'
🔔 Pulando atualização de bairro, pesquisa por cidade inteira.
Coletando informações do imóvel 1/1000...
Coletando informações do imóvel 2/1000...
Coletando informações do imóvel 3/1000...
Coletando informações do imóvel 4/1000...
Coletando informações do imóvel 5/1000...
Coletando informações do imóvel 6/1000...
Coletando informações do imóvel 7/1000...
Coletando informações do imóvel 8/1000...
Coletando informações do imóvel 9/1000...
Coletando informações do imóvel 10/1000...
Coletando informações do imóvel 11/1000...
Coletando informações do imóvel 12/1000...
Coletando informações do imóvel 13/1000...
Coletando informações do imóvel 14/1000...
Coletando informações do imóvel 15/1000...
Coletando 

Coletando informações do imóvel 182/1000...
Coletando informações do imóvel 183/1000...
Coletando informações do imóvel 184/1000...
Coletando informações do imóvel 185/1000...
Coletando informações do imóvel 186/1000...
Coletando informações do imóvel 187/1000...
Coletando informações do imóvel 188/1000...
Coletando informações do imóvel 189/1000...
Coletando informações do imóvel 190/1000...
Coletando informações do imóvel 191/1000...
Coletando informações do imóvel 192/1000...
Coletando informações do imóvel 193/1000...
Coletando informações do imóvel 194/1000...
Coletando informações do imóvel 195/1000...
Coletando informações do imóvel 196/1000...
Coletando informações do imóvel 197/1000...
Coletando informações do imóvel 198/1000...
Coletando informações do imóvel 199/1000...
Coletando informações do imóvel 200/1000...
Coletando informações do imóvel 201/1000...
Coletando informações do imóvel 202/1000...
Coletando informações do imóvel 203/1000...
Coletando informações do imóvel 

Coletando informações do imóvel 369/1000...
Coletando informações do imóvel 370/1000...
Coletando informações do imóvel 371/1000...
Coletando informações do imóvel 372/1000...
Coletando informações do imóvel 373/1000...
Coletando informações do imóvel 374/1000...
Coletando informações do imóvel 375/1000...
Coletando informações do imóvel 376/1000...
Coletando informações do imóvel 377/1000...
Coletando informações do imóvel 378/1000...
Coletando informações do imóvel 379/1000...
Coletando informações do imóvel 380/1000...
Coletando informações do imóvel 381/1000...
Coletando informações do imóvel 382/1000...
Coletando informações do imóvel 383/1000...
Coletando informações do imóvel 384/1000...
Coletando informações do imóvel 385/1000...
Coletando informações do imóvel 386/1000...
Coletando informações do imóvel 387/1000...
Coletando informações do imóvel 388/1000...
Coletando informações do imóvel 389/1000...
Coletando informações do imóvel 390/1000...
Coletando informações do imóvel 

Coletando informações do imóvel 556/1000...
Coletando informações do imóvel 557/1000...
Coletando informações do imóvel 558/1000...
Coletando informações do imóvel 559/1000...
Coletando informações do imóvel 560/1000...
Coletando informações do imóvel 561/1000...
Coletando informações do imóvel 562/1000...
Coletando informações do imóvel 563/1000...
Coletando informações do imóvel 564/1000...
Coletando informações do imóvel 565/1000...
Coletando informações do imóvel 566/1000...
Coletando informações do imóvel 567/1000...
Coletando informações do imóvel 568/1000...
Coletando informações do imóvel 569/1000...
Coletando informações do imóvel 570/1000...
Coletando informações do imóvel 571/1000...
Coletando informações do imóvel 572/1000...
Coletando informações do imóvel 573/1000...
Coletando informações do imóvel 574/1000...
Coletando informações do imóvel 575/1000...
Coletando informações do imóvel 576/1000...
Coletando informações do imóvel 577/1000...
Coletando informações do imóvel 

Coletando informações do imóvel 743/1000...
Coletando informações do imóvel 744/1000...
Coletando informações do imóvel 745/1000...
Coletando informações do imóvel 746/1000...
Coletando informações do imóvel 747/1000...
Coletando informações do imóvel 748/1000...
Coletando informações do imóvel 749/1000...
Coletando informações do imóvel 750/1000...
Coletando informações do imóvel 751/1000...
Coletando informações do imóvel 752/1000...
Coletando informações do imóvel 753/1000...
Coletando informações do imóvel 754/1000...
Coletando informações do imóvel 755/1000...
Coletando informações do imóvel 756/1000...
Coletando informações do imóvel 757/1000...
Coletando informações do imóvel 758/1000...
Coletando informações do imóvel 759/1000...
Coletando informações do imóvel 760/1000...
Coletando informações do imóvel 761/1000...
Coletando informações do imóvel 762/1000...
Coletando informações do imóvel 763/1000...
Coletando informações do imóvel 764/1000...
Coletando informações do imóvel 

Coletando informações do imóvel 930/1000...
Coletando informações do imóvel 931/1000...
Coletando informações do imóvel 932/1000...
Coletando informações do imóvel 933/1000...
Coletando informações do imóvel 934/1000...
Coletando informações do imóvel 935/1000...
Coletando informações do imóvel 936/1000...
Coletando informações do imóvel 937/1000...
Coletando informações do imóvel 938/1000...
Coletando informações do imóvel 939/1000...
Coletando informações do imóvel 940/1000...
Coletando informações do imóvel 941/1000...
Coletando informações do imóvel 942/1000...
Coletando informações do imóvel 943/1000...
Coletando informações do imóvel 944/1000...
Coletando informações do imóvel 945/1000...
Coletando informações do imóvel 946/1000...
Coletando informações do imóvel 947/1000...
Coletando informações do imóvel 948/1000...
Coletando informações do imóvel 949/1000...
Coletando informações do imóvel 950/1000...
Coletando informações do imóvel 951/1000...
Coletando informações do imóvel 

## Verificando se deu certo

In [4]:
df_imoveis.head()

,URL do anúncio,Descrição,Preço,Condomínio/IPTU,Info Geral,Endereço
0,https://www.quintoandar.com.br/imovel/89453991...,Apartamento à venda em Jardim Maria Estela de ...,R$ 200.000,R$ 98 Condo. + IPTU,65 m² · 4 quartos · 1 vaga,"Rua Professor Thomaz de Aquino, Jardim Maria E..."
1,https://www.quintoandar.com.br/imovel/89486607...,Apartamento à venda em Bom Retiro de 146 m².,R$ 290.000,R$ 1.762 Condo. + IPTU,146 m² · 3 quartos,"Rua Júlio Conceição339, Bom Retiro · São Paulo"
2,https://www.quintoandar.com.br/imovel/89486282...,Studio à venda em Paraíso de 21 m².,R$ 400.000,R$ 577 Condo. + IPTU,21 m² · 1 quarto,"Avenida Armando Ferrentini, Paraíso · São Paulo"
3,https://www.quintoandar.com.br/imovel/89438837...,Casa de 3 quartos à venda. Com 1 vaga na garagem.,R$ 270.000,R$ 0 Condo. + IPTU,85 m² · 3 quartos · 1 vaga,"Rua Filipe Nicoletti, Jardim Castelo · São Paulo"
4,https://www.quintoandar.com.br/imovel/89486499...,Apartamento de 2 quartos mobiliado à venda. Co...,R$ 1.200.000,R$ 1.110 Condo. + IPTU,65 m² · 2 quartos · 1 vaga,"Rua Ministro Luiz Gallotti, Vila Cordeiro · Sã..."


In [5]:
len(df_imoveis)

1000

## Tratando os dados

In [19]:
import pandas as pd
import re

df = df_imoveis.copy()

# 1. Reordenar as colunas para que "URL do anúncio" fique por último.
url = df.pop("URL do anúncio")
df["URL do anúncio"] = url

# 2. Tratar a coluna "Preço": remover "R$" e pontos, convertendo em valor numérico
df["Preço"] = (
    df["Preço"]
    .replace({r'R\$\s?': '', r'\.': ''}, regex=True)
    .astype(float)
)

# 3. Tratar a coluna "Condomínio/IPTU": remover "R$" e caracteres não numéricos
df["Condomínio/IPTU"] = (
    df["Condomínio/IPTU"]
    .replace({r'R\$\s?': '', r'\D': ''}, regex=True)
    .astype(float)
)

# 4. Extrair informações da coluna "Info Geral" e criar colunas separadas:
#    - Metragem quadrada (removendo o "m²")
#    - Número de quartos
#    - Número de vagas
def extrair_info(info):
    """
    Espera uma string no formato "116 m² · 3 quartos · 2 vagas"
    e retorna um tuple com (metragem, quartos, vagas).
    """
    partes = [parte.strip() for parte in info.split("·")]
    metragem = float(partes[0].replace("m²", "").strip()) if partes[0] else None
    quartos = int(re.search(r'\d+', partes[1]).group()) if len(partes) > 1 else None
    vagas   = int(re.search(r'\d+', partes[2]).group()) if len(partes) > 2 else None
    return metragem, quartos, vagas

df[['Metragem', 'Quartos', 'Vagas']] = df['Info Geral'].apply(lambda x: pd.Series(extrair_info(x)))

# 5. Separar a coluna "Endereço" em três novas colunas:
#    - Endereco_Rua
#    - Bairro
#    - Cidade
def extrair_endereco(endereco):
    """
    Espera uma string no formato: "Rua ..., Bairro · Cidade"
    e retorna (rua, bairro, cidade). 
    Caso o formato não esteja conforme, tenta extrair as partes disponíveis.
    """
    try:
        rua, resto = endereco.split(",", 1)
        if "·" in resto:
            bairro, cidade = resto.split("·", 1)
        else:
            bairro = resto
            cidade = None
    except ValueError:
        rua, bairro, cidade = endereco, None, None
    return rua.strip(), bairro.strip() if bairro else None, cidade.strip() if cidade else None

df[['Endereco_Rua', 'Bairro', 'Cidade']] = df['Endereço'].apply(lambda x: pd.Series(extrair_endereco(x)))

# 6. Criar novas colunas a partir da coluna "Descrição":
#    a) Tipo Imovel: identifica se é "Apartamento", "Casa" ou "Studio"
#    b) Mobiliado?: verifica se a palavra "mobiliado" existe na descrição

def identificar_tipo_imovel(descricao):
    # Lista de tipos a verificar
    tipos = ["Apartamento", "Casa", "Studio"]
    # Realiza a verificação de forma case insensitive
    for t in tipos:
        if t.lower() in descricao.lower():
            return t
    return "Sem informação"

# Aplica a função para identificar o tipo de imóvel
df["Tipo Imovel"] = df["Descrição"].apply(identificar_tipo_imovel)

# Cria a coluna "Mobiliado?" verificando se a palavra "mobiliado" está presente
df["Mobiliado?"] = df["Descrição"].apply(lambda x: "Sim" if "mobiliado" in x.lower() else "Sem informacao")

# Exibe as primeiras linhas do DataFrame resultante
df.head()

,Descrição,Preço,Condomínio/IPTU,Info Geral,Endereço,URL do anúncio,Metragem,Quartos,Vagas,Endereco_Rua,Bairro,Cidade,Tipo Imovel,Mobiliado?
0,Apartamento à venda em Jardim Maria Estela de ...,200000.0,98.0,65 m² · 4 quartos · 1 vaga,"Rua Professor Thomaz de Aquino, Jardim Maria E...",https://www.quintoandar.com.br/imovel/89453991...,65.0,4.0,1.0,Rua Professor Thomaz de Aquino,Jardim Maria Estela,São Paulo,Apartamento,Sem informacao
1,Apartamento à venda em Bom Retiro de 146 m².,290000.0,1762.0,146 m² · 3 quartos,"Rua Júlio Conceição339, Bom Retiro · São Paulo",https://www.quintoandar.com.br/imovel/89486607...,146.0,3.0,NaN,Rua Júlio Conceição339,Bom Retiro,São Paulo,Apartamento,Sem informacao
2,Studio à venda em Paraíso de 21 m².,400000.0,577.0,21 m² · 1 quarto,"Avenida Armando Ferrentini, Paraíso · São Paulo",https://www.quintoandar.com.br/imovel/89486282...,21.0,1.0,NaN,Avenida Armando Ferrentini,Paraíso,São Paulo,Studio,Sem informacao
3,Casa de 3 quartos à venda. Com 1 vaga na garagem.,270000.0,0.0,85 m² · 3 quartos · 1 vaga,"Rua Filipe Nicoletti, Jardim Castelo · São Paulo",https://www.quintoandar.com.br/imovel/89438837...,85.0,3.0,1.0,Rua Filipe Nicoletti,Jardim Castelo,São Paulo,Casa,Sem informacao
4,Apartamento de 2 quartos mobiliado à venda. Co...,1200000.0,1110.0,65 m² · 2 quartos · 1 vaga,"Rua Ministro Luiz Gallotti, Vila Cordeiro · Sã...",https://www.quintoandar.com.br/imovel/89486499...,65.0,2.0,1.0,Rua Ministro Luiz Gallotti,Vila Cordeiro,São Paulo,Apartamento,Sim


In [20]:
df = df.drop(columns=["Info Geral", "Endereço"])
#df.head()

In [21]:
# Tratando link
url = df.pop("URL do anúncio")
df["URL do anúncio"] = url
df['URL do anúncio'] = df['URL do anúncio'].str.extract(r'(https://www\.quintoandar\.com\.br/imovel/\d+/comprar/)')
df.head()

,Descrição,Preço,Condomínio/IPTU,Metragem,Quartos,Vagas,Endereco_Rua,Bairro,Cidade,Tipo Imovel,Mobiliado?,URL do anúncio
0,Apartamento à venda em Jardim Maria Estela de ...,200000.0,98.0,65.0,4.0,1.0,Rua Professor Thomaz de Aquino,Jardim Maria Estela,São Paulo,Apartamento,Sem informacao,https://www.quintoandar.com.br/imovel/89453991...
1,Apartamento à venda em Bom Retiro de 146 m².,290000.0,1762.0,146.0,3.0,NaN,Rua Júlio Conceição339,Bom Retiro,São Paulo,Apartamento,Sem informacao,https://www.quintoandar.com.br/imovel/89486607...
2,Studio à venda em Paraíso de 21 m².,400000.0,577.0,21.0,1.0,NaN,Avenida Armando Ferrentini,Paraíso,São Paulo,Studio,Sem informacao,https://www.quintoandar.com.br/imovel/89486282...
3,Casa de 3 quartos à venda. Com 1 vaga na garagem.,270000.0,0.0,85.0,3.0,1.0,Rua Filipe Nicoletti,Jardim Castelo,São Paulo,Casa,Sem informacao,https://www.quintoandar.com.br/imovel/89438837...
4,Apartamento de 2 quartos mobiliado à venda. Co...,1200000.0,1110.0,65.0,2.0,1.0,Rua Ministro Luiz Gallotti,Vila Cordeiro,São Paulo,Apartamento,Sim,https://www.quintoandar.com.br/imovel/89486499...


In [22]:
df['URL do anúncio'][30]

'https://www.quintoandar.com.br/imovel/894859285/comprar/'

## Exportar para Excel

In [24]:
nome_arquivo = "imoveis_quintoandar.xlsx"
df.to_excel(nome_arquivo, index=False)

print(f"📁 Arquivo Excel salvo com sucesso como: {nome_arquivo}")

📁 Arquivo Excel salvo com sucesso como: imoveis_quintoandar.xlsx
